<a href="https://colab.research.google.com/github/D-Barradas/RAPIDS_HPO/blob/main/notebooks/HPO_Skorch_RAPIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Required Libraries
We start by installing and importing all the necessary libraries for this tutorial. This includes RAPIDS libraries (cuDF, cuML), PyTorch, Skorch, and other utilities for data handling and visualization.

In [ ]:
!pip install skorch
!pip install scikit-learn pandas
!pip install optuna
!pip install optuna-integration[skorch]
!pip3 install torch torchaudio torchvision torchtext torchdata


In [ ]:
import warnings
warnings.filterwarnings('ignore') # Reduce number of messages/warnings displayed

### Evaluate Model Performance
After training, we evaluate the model's performance on the test set. We display key metrics such as accuracy and visualize the results to better understand the model's effectiveness.

# Hyperparameter Optimization (HPO) with Skorch and RAPIDS
This tutorial demonstrates how to use RAPIDS for GPU-accelerated data processing and Skorch for PyTorch-based models with scikit-learn compatible HPO tools.


## 1. Import Required Libraries
Import RAPIDS libraries (cuDF, cuML), PyTorch, Skorch, and any other required libraries.


In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
import cudf
import cupy as cp
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from cuml.datasets import make_classification


## 2. Set Up RAPIDS Environment
Configure the RAPIDS environment and check GPU availability.

### Set Up RAPIDS Environment
Here, we configure the RAPIDS environment and check if a GPU is available. This ensures that all RAPIDS and PyTorch operations will run on the GPU for maximum acceleration.

In [ ]:
# Check GPU availability
import torch
print('CUDA available:', torch.cuda.is_available())

import cuml
print('cuML version:', cuml.__version__)

import cudf
print('cuDF version:', cudf.__version__)

## 3. Prepare Dataset
Load a dataset using cuDF, perform preprocessing, and split into training and test sets.


We generate a synthetic classification dataset using RAPIDS cuML, convert it to numpy arrays for PyTorch/Skorch compatibility, and split it into training and test sets. This step demonstrates RAPIDS-accelerated data handling.

In [ ]:
# Generate synthetic data on GPU with cuML
# You can change the n_samples and n_features to stress test the HPO algorithm
df_X, df_y = make_classification(n_samples=10000, n_features=20, n_informative=15, n_classes=2, random_state=42)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y , test_size=0.2, random_state=42)

## 4. Define Neural Network with Skorch
Create a PyTorch neural network model and wrap it with Skorch for scikit-learn compatibility.

### Define Neural Network with Skorch
We define a simple feedforward neural network in PyTorch and wrap it with Skorch's `NeuralNetClassifier`. This makes the model compatible with scikit-learn tools and enables easy integration with hyperparameter optimization libraries.

In [ ]:
class ClassifierModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super().__init__()
        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, 2)
    def forward(self, X):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.dense1(X)
        return X

net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=10,
    lr=0.1,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

## 5. Train Model Using RAPIDS Data Structures
Train the Skorch model using RAPIDS-accelerated data structures and monitor training progress. We'll use GridSearchCV for HPO.

### Train Model Using RAPIDS Data Structures
We train the Skorch-wrapped PyTorch model using the RAPIDS-accelerated data structures. Training progress and metrics are monitored to ensure the model is learning effectively.

In [ ]:
params = {
    'lr': [0.01, 0.1],
    'max_epochs': [10, 20],
    'module__num_units': [10, 20], # this parameter is inside of the ClassifierModule also this is Neurons in each layer
}

gs = GridSearchCV(net, params, refit=True, cv=3, scoring='accuracy', verbose=2)
# the property .get() is used to retrieve the underlying data from the cuDF DataFrame and convert it to a NumPy array
gs.fit(X_train.get(), y_train.get())

## 6. Evaluate Model Performance
Evaluate the trained model on the test set and display performance metrics.

In [ ]:
print('Best parameters:', gs.best_params_)
print('Best CV accuracy:', gs.best_score_)
test_acc = gs.score(X_test.get(), y_test.get())
print('Test accuracy:', test_acc)

# Visualize grid search results
import pandas as pd
results = pd.DataFrame(gs.cv_results_)
pivot = results.pivot_table(index='param_module__num_units', columns='param_lr', values='mean_test_score')
pivot.plot(kind='bar')
plt.ylabel('Mean CV Accuracy')
plt.title('Grid Search Results')
plt.show()

# Hyperparameter Optimization with Optuna and Skorch (RAPIDS Accelerated)
This section demonstrates how to use Optuna for hyperparameter optimization of a Skorch-wrapped PyTorch model, leveraging RAPIDS for data handling and GPU acceleration.

### Install and Import Optuna
We install Optuna (if not already installed) and import the necessary modules, including the SkorchPruningCallback. This callback allows Optuna to stop unpromising trials early, saving compute time.

In [ ]:
import optuna
from optuna.integration import SkorchPruningCallback

### Define a Flexible PyTorch Model
We define a PyTorch neural network class (`OptunaClassifierModule`) whose architecture (number of layers, units per layer, dropout) can be controlled by hyperparameters. This flexibility allows Optuna to search over different model architectures.

In [ ]:
class OptunaClassifierModule(nn.Module):
    def __init__(self, n_layers, dropout, hidden_units):
        super().__init__()
        layers = []
        input_dim = 20  # matches synthetic dataset features
        for i in range(n_layers):
            layers.append(nn.Linear(input_dim, hidden_units[i]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_dim = hidden_units[i]
        layers.append(nn.Linear(input_dim, 2))
        self.model = nn.Sequential(*layers)
    def forward(self, x):
        return self.model(x)

### Define the Optuna Objective Function
The `optuna_objective` function is called by Optuna for each trial. It:
- Suggests hyperparameters (number of layers, units, dropout, learning rate, epochs)
- Instantiates the model and wraps it with Skorch's `NeuralNetClassifier`
- Trains the model on the training set
- Evaluates accuracy on the test set
- Returns the accuracy as the objective to maximize

The SkorchPruningCallback is used to enable early stopping of bad trials.

In [ ]:
def optuna_objective(trial, X_train, X_test, y_train, y_test):
    n_layers = trial.suggest_int('n_layers', 1, 3)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    hidden_units = [trial.suggest_int(f'n_units_l{i}', 16, 128) for i in range(n_layers)] # Neurons in each layer
    max_epochs = trial.suggest_int('max_epochs', 10, 30)
    lr = trial.suggest_float('lr', 1e-4, 1e-1, log=True)

    model = OptunaClassifierModule(n_layers, dropout, hidden_units)
    net = NeuralNetClassifier(
        model,
        criterion=nn.CrossEntropyLoss,
        max_epochs=max_epochs,
        lr=lr,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        verbose=0,
        callbacks=[SkorchPruningCallback(trial, 'valid_acc')],
    )
    net.fit(X_train.astype(np.float32), y_train)
    y_pred = net.predict(X_test.astype(np.float32))
    return (y_pred == y_test).mean()

### Run the Optuna Study
We create an Optuna study and run the optimization for a set number of trials or until a timeout. Optuna will try different hyperparameter combinations, train the model, and keep track of the best results.

In [ ]:
# Run Optuna HPO

# This line tells Optuna: “Use the MedianPruner to automatically stop trials that are not performing well,
#  so we don’t waste time on bad hyperparameter combinations.”

pruner = optuna.pruners.MedianPruner()

# This line creates an Optuna study that will search for the best hyperparameters to maximize your chosen metric,
#  and will use the pruner to stop bad trials early. The study object will manage all the optimization work for you.

study = optuna.create_study(direction='maximize', pruner=pruner)

# The Following line tells Optuna: “Try up to 20 different sets of hyperparameters (or stop after 10 minutes). 
# For each set, call the optuna_objective function with the current trial and the training/testing data. Use the results to find the best model.”
# The lambda is just a quick way to pass the trial and data to your objective function.

study.optimize(lambda trial: optuna_objective(trial, X_train.get(), X_test.get(), y_train.get(), y_test.get()), n_trials=20, timeout=600)

print(f"Number of finished trials: {len(study.trials)}")
print(f"Best trial value: {study.best_trial.value}")
print("Best trial parameters:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")

### Review the Results
After optimization, we print the number of trials, the best accuracy found, and the best hyperparameters. This helps us understand which model configuration performed best on our data.

In [ ]:
# Visualize Optuna optimization history and parameter importances
import optuna.visualization as vis

# Plot optimization history
vis.plot_optimization_history(study).show()

# Plot parameter importances
vis.plot_param_importances(study).show()